<a href="https://colab.research.google.com/github/sundarjhu/EscueladeVerano_2021/blob/main/Escuela_de_Verano_2021_Demo_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Accessing data from published tables on Vizier**
---
### Vizier consists of a collection of tables published as part of individual papers, which are accessed using their unique identifiers.


---



In [2]:
from astropy.table import Table, join
import numpy as np
!pip install pyvo
import pyvo as vo

     |████████████████████████████████| 802 kB 5.2 MB/s 
  Created wheel for mimeparse: filename=mimeparse-0.1.3-py3-none-any.whl size=3864 sha256=5ded9bdbf5c1b7ee70ccfd6d8caea1832fe651d9ccf2b8f71b36a1472d28fd30
  Stored in directory: /root/.cache/pip/wheels/49/b4/2d/0081759ae1833bd694024801f7aacddcda8a687e8d5fbaeebd
Successfully built mimeparse


### The Vizier TAP service is available through two urls. One of the them is the backup url, which doesn't accept upload tables. We will avoid that one below.

In [3]:
r = vo.regsearch(servicetype = 'tap', keywords = ['Vizier'])
# There are two servers, the backup server doesn't allow uploads. Choose the correct one.
k = int(np.nonzero(['obstap' not in rr['ivoid'] for rr in r])[0]) # avoid the column corresponding to the url that doesn't accept uploads
Vizier_service = r[k].service

### A basic query to a published table with a known identifier (obtained from the ADS entry) is easy:

In [4]:
# Select the top 10 rows from the Abrahamyan et al. 2015 table
# Demonstration: go to ADS and find the links to the Vizier catalogue paper by the author. Follow the CDS link to find the table identifier.
query = """
  SELECT top 10 * from "II/328/catalog"
"""

###We can automate this further by querying for the table identifier.
###This information is available in the Vizier service in a table named, imaginatively, `table`.

In [18]:
# Find a catalogue using the `tables` table and its `description` column
query = """
  SELECT * FROM tables 
  WHERE description LIKE '%Abrahamyan%' and description LIKE '%IRAS%PSC%FSC%'
"""
t = Vizier_service.search(query).to_table()
t.pprint_all()
print("*******************")

# Now query the table with that unique identifier
query = """ SELECT top 10 * from """ + '"' + t['table_name'][0] + '"'
r = Vizier_service.search(query).to_table()
r.pprint_all()

schema_name   table_name   table_type                                      description                                       utype
----------- -------------- ---------- -------------------------------------------------------------------------------------- -----
       viz7 II/338/catalog      table The IRAS PSC/FSC Combined Catalogue ( Abrahamyan H.V., Mickaelian A.M., Knyazyan A.V.)      
*******************
recno  IRAS-PSC    RApsc    DEpsc   pscMaj pscMin pscPA q_psc12 psc12 q_psc25 psc25 q_psc60 psc60 q_psc100 psc100   IRAS-FSC    RAfsc    DEfsc   fscMaj fscMin fscPA q_fsc12 fsc12  q_fsc25 fsc25 q_fsc60 fsc60 q_fsc100 fsc100    RAir    e_RAir    DEir    e_DEir P/F irMaj  irMin  irPA d(ir)  k(ir) Fir12  Fir25 Fir60 Fir100 f_ir  cl   Akari-IRC     RAirc   DEirc_deg ircMaj ircMin ircPA d(irc) k(irc) q_S09  S09    e_S09  q_S18  S18    e_S18   r09    r18     Akari-FIS     RAfis  DEfis  fisErr d(fis) k(fis) q_S65  S65  e_S65 q_S90  S90  e_S90  q_S140  S140 e_S140 q_S160 S160 e_S160      

##Put it all together

### Find a catalogue using the `tables` table and its `description` column

In [53]:

query = """
  SELECT * FROM tables 
  WHERE description LIKE '%Abrahamyan%' and description LIKE '%IRAS%PSC%FSC%'
"""
table_id = Vizier_service.search(query).to_table()

### Only select those rows in the above table that have valid IRAS PSC identifiers, and only those rows with valid fluxes in all four IRAS bands.
### We will retrieve only the coordinates and the IRAS PSC fluxes.
### Notice the usage of `!= ''` for the string field and `not NULL` for the float field


In [36]:
query = """ SELECT top 1000 "IRAS-PSC", RApsc, DEpsc, psc12, psc25, psc60, psc100 
            FROM """ + '"' + table_id['table_name'][0] + '"' + """ 
            WHERE ("IRAS-PSC" != '') and (psc12 is not NULL) and (psc25 is not NULL) and (psc60 is not NULL) and (psc100 is not NULL) """
results = Vizier_service.search(query).to_table()
results.pprint()

### We need to convert these fluxes to magnitudes using the flux zeropoints (zero-magnitude fluxes) in the Vega system.
### These can be obtained from the Spanish Virtual Observatory website.


In [ ]:
import subprocess
from astropy.io.votable import parse

filternames = ['IRAS/IRAS.12mu', 'IRAS/IRAS.25mu', 'IRAS/IRAS.60mu', 'IRAS/IRAS.100mu']
url = 'http://svo2.cab.inta-csic.es//theory/fps3/fps.php?ID='
fZP = [] # Vega zeropoint fluxes for the IRAS filters
for filtername in filternames:
  # Download the filter table from the SVO database
  _ = subprocess.call(['curl', '-o', 'temp.vot', url + filtername])
  # Parse the resulting VOTable and access the ZeroPoint PARAM in the header
  v = parse('temp.vot')
  fZP.append(v.get_field_by_id('ZeroPoint').value)
_ = subprocess.call(['rm', 'temp.vot'])

# Use these flux zeropoints to convert the IRAS PSC fluxes into magnitudes
fluxcols = ['psc12', 'psc25', 'psc60', 'psc100']
magcols = ['mag12', 'mag25', 'mag60', 'mag100']
for i in range(4):
  r[magcols[i]] = -2.512 * np.log10(r[fluxcols[i]] / np.repeat(fZP[i], len(r)))

# An IRAS colour-magnitude diagram from these magnitudes
import matplotlib.pyplot as plt
plt.figure(figsize = (10, 10))
plt.plot(r['mag12'] - r['mag25'], r['mag60'] - r['mag100'], 'ro')
plt.xlabel('[12] - [25] (mag)', fontdict = {'size': 16, 'weight': 'bold'})
plt.ylabel('[25] - [60] (mag)', fontdict = {'size': 16, 'weight': 'bold'})
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.show()